In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
from datetime import datetime
import pytz

In [5]:
sw_lat, sw_long = 38.757, -94.908
ne_lat, ne_long = 39.427, -94.235
query = f"""
[out:json];
(
  node["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
  way["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
  relation["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
);
out center;
"""

response = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
data = response.json()
for el in data["elements"]:
  name = el.get("tags", {}).get("name")
  lat = el.get("lat")
  lon = el.get("lon")
  # if name != None:
  #   print(f'{name}: {lat}, {lon}')
  print(el)

{'type': 'node', 'id': 2838921442, 'lat': 39.4213658, 'lon': -94.5476547, 'tags': {'leisure': 'golf_course', 'name': 'Posse Course'}}
{'type': 'node', 'id': 2882365085, 'lat': 38.986333, 'lon': -94.5238809, 'tags': {'leisure': 'golf_course', 'name': 'The Hill Course'}}
{'type': 'node', 'id': 2882365086, 'lat': 38.9893881, 'lon': -94.5222873, 'tags': {'leisure': 'golf_course', 'name': 'The Rock Course'}}
{'type': 'node', 'id': 2882365087, 'lat': 38.9919431, 'lon': -94.5278009, 'tags': {'leisure': 'golf_course', 'name': 'The River Course'}}
{'type': 'node', 'id': 6107100689, 'lat': 39.4232906, 'lon': -94.5514754, 'tags': {'addr:city': 'Smithville', 'addr:postcode': '64089', 'addr:state': 'MO', 'addr:street': 'Golf Course Road', 'leisure': 'golf_course', 'name': 'Paradise Pointe', 'opening_hours': 'Mo-Su 08:00-17:00'}}
{'type': 'node', 'id': 10853398897, 'lat': 39.1671392, 'lon': -94.7610853, 'tags': {'leisure': 'golf_course', 'name': 'Brough Creek National GC'}}
{'type': 'way', 'id': 384

In [ ]:
class GolfCourse:
  def __init__(self, name, lat, lon):
    self.name = name
    self.lat = lat
    self.lon = lon

  def __repr__(self):
    return f"GolfCourse(name={self.name}, lat={self.lat}, lon={self.lon})"

In [2]:
bookateetime_courses = {
  'Shoal Creek Golf Course': '118-1',
  'Hodge Park Golf Course': '117-1',
  'Winterstone Golf Course': '62-1',
  'Adams Pointe Golf Course': '45-1',
  'Heart of America Golf Course': '49-1',
  'Sycamore Ridge Golf Club': '44-1',
  'Paradise Pointe Golf Course - The Outlaw': '24-1',
  'Paradise Pointe Golf Course - The Posse': '24-2',
}

chronogolf_courses = {
  'Falcon Lakes Golf Club': 6633,
}

golfback_courses = {
  "Dub's Dread Golf Club": '398d44ce-a908-4ce7-8f50-e5f4bdc77b73',
  'Painted Hills Golf Club': '857a12d4-a9cf-4a43-afe2-60940bdc7438',
  'Drumm Farm Golf Club - Full': 'd70999c9-d7d4-4008-9f41-4e9551b3c796',
  'Drumm Farm Golf Club - Executive': '9a1de435-8a46-4840-9cdc-332c3cfea782',
  'Royal Meadows Golf Club': 'd2278228-4700-4354-95a8-422a8f9a5a16'
}

foreup_courses = {
  'Teetering Rocks Golf Course': 7341,
  'Heritage Park Golf Course': 12159,
  'Tomahawk Hills Golf Course': 11026,
}

loners = {
  'Sunflower Hills Golf Course': 'https://www.sunflowerhillsgolfcourse.com/TeeTimes',
}

In [ ]:
date = '2025-04-30'
players = 4

tee_times_df = pd.DataFrame()

for course in bookateetime_courses.items():
  # Make a GET request to the URL with the course name and date
  response = requests.get(f"https://bookateetime.teequest.com/search/{course[1]}/{date}?selectedPlayers={players}&selectedHoles=18")

  # Decode the response content as a string
  content_str = response.content.decode('utf-8')
  soup = BeautifulSoup(content_str, 'html.parser')
  tee_times = []

  for tee_time_div in soup.find_all('div', class_='tee-time'):
    href = tee_time_div.find('a', class_='btn')['href']
    tt = {
      'course': course[0],
      'tee_time': tee_time_div['data-date-time'],
      'price': float(tee_time_div['data-price']),
      'players': int(tee_time_div['data-available']),
      'book_url': f'https://bookateetime.teequest.com{href}'
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)

# Convert the 'date_time' column to datetime format
tee_times_df['tee_time'] = pd.to_datetime(tee_times_df['tee_time'], format='%Y%m%d%H%M')

In [50]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Show full column content

tee_times_df[
  tee_times_df['course'] == 'Paradise Pointe Golf Course - The Posse'
].sort_values('tee_time').head(10)

,course,tee_time,price,players,book_url
86,Paradise Pointe Golf Course - The Posse,2025-04-30 09:00:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504300900/1/A/AO/4/18
87,Paradise Pointe Golf Course - The Posse,2025-04-30 10:40:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301040/1/A/AO/4/18
88,Paradise Pointe Golf Course - The Posse,2025-04-30 11:40:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301140/1/A/AO/4/18
89,Paradise Pointe Golf Course - The Posse,2025-04-30 11:50:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301150/1/A/AO/4/18
90,Paradise Pointe Golf Course - The Posse,2025-04-30 12:10:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301210/1/A/AO/4/18
91,Paradise Pointe Golf Course - The Posse,2025-04-30 12:40:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301240/1/A/AO/4/18
92,Paradise Pointe Golf Course - The Posse,2025-04-30 12:50:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301250/1/A/AO/4/18
93,Paradise Pointe Golf Course - The Posse,2025-04-30 13:00:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301300/1/A/AO/4/18
94,Paradise Pointe Golf Course - The Posse,2025-04-30 13:10:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301310/1/A/AO/4/18
95,Paradise Pointe Golf Course - The Posse,2025-04-30 13:20:00,61.0,4,https://bookateetime.teequest.com/teetime/24-2/202504301320/1/A/AO/4/18


In [ ]:
for course in golfback_courses.items():
  url = f"https://api.golfback.com/api/v1/courses/{course[1]}/date/{date}/teetimes"
  headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://golfback.com/",
    "Content-Type": "application/json",
  }
  params = {
    "date": date,
    "course_id": course[1],
    "players": players
  }

  response = requests.post(url, headers=headers, json=params)
  tee_times_raw = response.json()['data']
  
  tee_times = []
  for tee_time in tee_times_raw:
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time['dateTime'], format='%Y-%m-%dT%H:%M:%S%z') \
        .astimezone(pytz.timezone("US/Central")).strftime("%Y-%m-%d %H:%M:%S"),
      'price': float(tee_time['rates'][0]['price']),
      'players': tee_time['playersMax'],
      'book_url': f'https://golfback.com/#/course/{course[1]}/date/{date}/teetime/{tee_time['id']}?rateId={tee_time['rates'][0]['ratePlanId']}&holes=18&players={players}'
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)


{'id': '995bad5b-7312-4987-977f-b0d12d5ab3ba', 'courseId': '398d44ce-a908-4ce7-8f50-e5f4bdc77b73', 'courseName': "Dub's Dread Golf Course", 'dateTime': '2025-04-30T07:30:00-05:00', 'localDateTime': '2025-04-30T07:30:00', 'rates': [{'ratePlanId': 'df4e91ee-cb65-4ca6-a2f3-eb57a2044bc8', 'name': 'Public 18', 'description': None, 'holes': 18, 'hasCartIncluded': True, 'isPrimary': True, 'usePrimaryAfterSelection': False, 'isDeal': False, 'isGimme': False, 'basePrice': 48.0, 'price': 48.0, 'feeDisplay': 0.0}], 'isAvailable': True, 'holes': [18], 'has9Holes': False, 'hasDeal': False, 'primaryPrices': [{'holes': 18, 'basePrice': 48.0, 'price': 48.0}], 'playersMin': 2, 'playersMax': 4, 'location': None, 'lockExpiration': None, 'playersDisplay': '2&nbsp;-&nbsp;4'}
https://golfback.com/#/course/398d44ce-a908-4ce7-8f50-e5f4bdc77b73/date/2025-04-30/teetime/995bad5b-7312-4987-977f-b0d12d5ab3ba?rateId=df4e91ee-cb65-4ca6-a2f3-eb57a2044bc8&holes=18&players=4
{'id': '929fa58a-c2d5-4567-b415-4efef8a497a0

In [ ]:
for course in foreup_courses.items():
  flip_date = datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y')
  url = f"https://foreupsoftware.com/index.php/api/booking/times?time=all&date={flip_date}&holes=all&players={players}&booking_class=14824&schedule_id={course[1]}&api_key=no_limits"
  headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": f"https://foreupsoftware.com/index.php/booking/22857/7340",
    "Content-Type": "application/json",
  }
  params = {
    "date": datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y'),
    "players": players,
  }

  response = requests.get(url, headers=headers, json=params)
  tee_times_raw = response.json()

  tee_times = []
  for tee_time in tee_times_raw:
    # print(tee_time)
    # break
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time['time'], format='%Y-%m-%d %H:%M'),
      'price': float(tee_time['green_fee'] + tee_time['cart_fee']),
      'players': tee_time['available_spots']
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)


{'time': '2025-04-30 08:00', 'start_front': 202503300800, 'course_id': 21235, 'course_name': 'Teetering Rocks Links', 'schedule_id': 7341, 'teesheet_id': 7341, 'schedule_name': 'Teetering Rocks Links', 'require_credit_card': False, 'teesheet_holes': 18, 'teesheet_side_id': 6231, 'teesheet_side_name': 'Front', 'teesheet_side_order': 1, 'reround_teesheet_side_id': 6232, 'reround_teesheet_side_name': 'Back', 'available_spots': 4, 'available_spots_9': 4, 'available_spots_18': 4, 'maximum_players_per_booking': '4', 'minimum_players': '1', 'allowed_group_sizes': ['1', '2', '3', '4'], 'holes': '9/18', 'has_special': False, 'special_id': False, 'special_discount_percentage': 0, 'group_id': False, 'booking_class_id': 14824, 'booking_fee_required': False, 'booking_fee_price': False, 'booking_fee_per_person': False, 'foreup_trade_discount_rate': 0, 'trade_min_players': 8, 'trade_cart_requirement': 'riding', 'trade_hole_requirement': '18', 'trade_available_players': 0, 'green_fee_tax_rate': False,

In [ ]:
tee_times_df.head()

In [56]:
tee_times_df['tee_time'] = pd.to_datetime(tee_times_df['tee_time'], utc=True)

In [53]:
tee_times_df[
  (tee_times_df['tee_time'].dt.hour >= 16) &
  (tee_times_df['price'] <= 90)
].sort_values(by=['course', 'tee_time'])#.to_csv('tee_times.csv', index=False)

AttributeError: Can only use .dt accessor with datetimelike values

In [19]:
tee_times_df.sort_values(by=['course', 'tee_time']).to_csv('tee_times.csv', index=False)